In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import os
os.chdir('../')
os.environ["CUDA_VISIBLE_DEVICES"]="0"
os.environ["PROTOCOL_BUFFERS_PYTHON_IMPLEMENTATION"]="python"


In [ ]:
from src.dataset import *
from src.concept_vectors import *
from src.util import *
from src.hierarchy import *
from src.metrics import *
import numpy as np
import matplotlib.pyplot as plt
import glob
import tensorflow as tf
from src.create_vectors import *
import json 

In [ ]:
import os 
os.system("export LD_LIBRARY_PATH=/usr/local/cuda/lib64:$LD_LIBRARY_PATH")

In [ ]:
seeds = [43,44,45]
vector_method = [load_cem_vectors_simple,load_concept2vec_vectors_simple,load_label_vectors_simple,load_tcav_vectors_simple]
vector_names = ["CEM","Concept2Vec","Label","TCAV"]

## Debugging: Check which TCAV Vectors are available

In [ ]:
dataset = CUB_Dataset()
attributes = dataset.get_attributes()

In [ ]:
def attribute_exists(attribute_name,suffix,seed):
    attribute_name = attribute_name.replace(" ","_")
    folder_name = "results/bases/tcav/{}{}/{}/".format(dataset.experiment_name,suffix,seed)
    for i in range(3):
        file_name = "{}_{}_{}-random500_{}-block4_conv1-linear-0.1.pkl".format(attribute_name,seed,suffix,i)

        if not os.path.exists(folder_name + file_name):
            return False
    return True

In [ ]:
for attribute in attributes:
    for seed in [43,44,45]:
        for suffix in ['','_image_robustness','_image_responsiveness']:
            attribute_exists(attribute,suffix,seed)

## Evaluate All Vectors MNIST

In [ ]:
dataset = MNIST_Dataset()
attributes = dataset.get_attributes()

In [ ]:
for method,name in zip(vector_method,vector_names):
    print("Computing {}".format(name))
    start = time.time()

    results = compute_all_metrics(method,
                                        dataset,
                                        attributes,
                                        seeds)
    print("Method {}: {}".format(name,results))
    
    name_lower = name.lower()
    w = open("results/evaluation/{}_{}.txt".format(dataset.experiment_name,name_lower),"w")
    for key in results:
        w.write("{}: {}\n".format(key,results[key]))
    w.close()


In [ ]:
baseline_distances = np.zeros((len(attributes),len(attributes)))
for i, attribute_1 in enumerate(attributes):
    for j, attribute_2 in enumerate(attributes):
        baseline_distances[i][j] = (1-int(attribute_1[0] == attribute_2[0]))

In [ ]:
distance_by_method_mnist = {}
for function,name in zip(vector_method,vector_names):
    h_list = [flat_distance_to_square(get_concept_distances(function,MNIST_Dataset(),'',MNIST_Dataset().get_attributes(),seed)) for seed in [43,44,45]]
    distance_by_method_mnist[name] = [embedding_distance(h,baseline_distances,k=1) for h in h_list]
    distance_by_method_mnist[name] = (np.mean(distance_by_method_mnist[name]),np.std(distance_by_method_mnist[name]))

In [ ]:
json.dump(distance_by_method_mnist,open('results/evaluation/ablation/distance_mnist.json','w'))

In [ ]:
method_list = [load_label_vectors_simple,load_cem_vectors_simple,load_concept2vec_vectors_simple,load_tcav_vectors_simple]
name_list = ['label','cem','concept2vec','tcav']
dataset = MNIST_Dataset()

agreement_by_method = {}

for method,name in zip(method_list,name_list):
    agreement_by_method[name] = []

    for seed in [43,44,45]:
        all_vectors = [np.mean(np.array(method(a,dataset,"",seed)),axis=0) for a in dataset.get_attributes()]
        all_vectors = np.array(all_vectors)

        closest_vectors = []

        # Iterate through each vector in the array
        for i in range(len(all_vectors)):
            current_vector = all_vectors[i]
            
            # Compute cosine similarity with all other vectors
            similarities = [1 - cosine(current_vector, other_vector) for other_vector in all_vectors]

            # Find the index of the vector with the maximum cosine similarity (excluding the current vector)
            closest_index = np.argmax(similarities[:i] + similarities[i+1:])  # Exclude the current vector
            
            if closest_index >= i:
                closest_index += 1

            # Append the closest vector to the list
            closest_vectors.append(closest_index)

        # Convert the list to a NumPy array if needed
        closest_vectors = np.array(closest_vectors)
        correct_vectors = []

        for i in range(0,len(dataset.get_attributes()),2):
            correct_vectors.append(i+1)
            correct_vectors.append(i)
        
        agreement_by_method[name].append(float(np.sum(np.array(closest_vectors) == np.array(correct_vectors))/len(correct_vectors)))
json.dump(agreement_by_method,open('results/evaluation/ablation/agreement_mnist.json','w'))

## Evaluate all Vectors CUB

In [ ]:
dataset = CUB_Dataset()
attributes = dataset.get_attributes()
seeds = [43,44,45]

In [ ]:
for method,name in zip(vector_method[::-1],vector_names[::-1]):
    print("Computing {}".format(name))
    start = time.time()

    results = compute_all_metrics(method,
                                        dataset,
                                        attributes,
                                        seeds)
    print("Method {}: {}".format(name,results))
    
    name_lower = name.lower()
    w = open("results/evaluation/{}_{}.txt".format(dataset.experiment_name,name_lower),"w")
    for key in results:
        w.write("{}: {}\n".format(key,results[key]))
    w.close()
    

In [ ]:
baseline_distances_color = np.zeros((len(attributes),len(attributes)))
for i, attribute_1 in enumerate(attributes):
    for j, attribute_2 in enumerate(attributes):
        baseline_distances_color[i][j] = (1-int(attribute_1.split("::")[1] == attribute_2.split("::")[1])) + abs(random.random()/100)*int(attribute_1 != attribute_2)

In [ ]:
distance_by_method_cub = {}
for function,name in zip(vector_method,vector_names):
    h_list = [flat_distance_to_square(get_concept_distances(function,dataset,'',attributes,seed)) for seed in [43,44,45]]
    distance_by_method_cub[name] = [embedding_distance(h,baseline_distances_color,k=3) for h in h_list]
    distance_by_method_cub[name] = (np.mean(distance_by_method_cub[name]),np.std(distance_by_method_cub[name]))

In [ ]:
json.dump(distance_by_method_cub,open('results/evaluation/ablation/distance_cub_second_part.json','w'))

In [ ]:
top_pairs = get_top_k_pairs(baseline_distances_color,k=3)
top_pairs = [(attributes[i[0]],attributes[i[1]]) for i in top_pairs]

In [ ]:
our_top_pairs = get_top_k_pairs(flat_distance_to_square(get_concept_distances(load_label_vectors_simple,dataset,'',attributes,43)))

In [ ]:
our_top_pairs = [(attributes[i[0]],attributes[i[1]]) for i in our_top_pairs]

In [ ]:
baseline_distances_attribute = np.zeros((len(attributes),len(attributes)))

In [ ]:
for i, attribute_1 in enumerate(attributes):
    for j, attribute_2 in enumerate(attributes):
        baseline_distances_attribute[i][j] = (1-int(attribute_1.split("::")[0] == attribute_2.split("::")[0])) + abs(np.random.random()/100)*int(attribute_1 != attribute_2)

In [ ]:
distance_by_method_cub = {}
for function,name in zip(vector_method,vector_names):
    h_list = [flat_distance_to_square(get_concept_distances(function,dataset,'',attributes,seed)) for seed in [43,44,45]]
    distance_by_method_cub[name] = [embedding_distance(h,baseline_distances_attribute,k=3) for h in h_list]
    distance_by_method_cub[name] = (np.mean(distance_by_method_cub[name]),np.std(distance_by_method_cub[name]))

In [ ]:
json.dump(distance_by_method_cub,open('results/evaluation/ablation/distance_cub_first_part.json','w'))

## Evalaute all Vectors DSprites

In [ ]:
dataset = DSprites_Dataset()
attributes = dataset.get_attributes()

In [ ]:
for method,name in zip(vector_method[::-1],vector_names[::-1]):
    print("Computing {}".format(name))
    start = time.time()

    results = compute_all_metrics(method,
                                        dataset,
                                        attributes,
                                        seeds)
    print("Method {}: {}".format(name,results))
    
    name_lower = name.lower()
    w = open("results/evaluation/{}_{}.txt".format(dataset.experiment_name,name_lower),"w")
    for key in results:
        w.write("{}: {}\n".format(key,results[key]))
    w.close()

## Evaluate all Vectors Chexpert

In [ ]:
dataset = Chexpert_Dataset()
attributes = dataset.get_attributes()

In [ ]:
for method,name in zip(vector_method,vector_names):
    print("Computing {}".format(name))
    start = time.time()

    results = compute_all_metrics(method,
                                        dataset,
                                        attributes,
                                        seeds)
    print("Method {}: {}".format(name,results))
    
    name_lower = name.lower()
    w = open("results/evaluation/{}_{}.txt".format(dataset.experiment_name,name_lower),"w")
    for key in results:
        w.write("{}: {}\n".format(key,results[key]))
    w.close()

## Explain why CEM Vectors are Random

In [ ]:
results_by_method = {'cem': {}, 
'tcav': {}, 
'label': {}, 
'concept2vec': {}}

for method,name in zip([load_cem_vectors_simple,load_tcav_vectors_simple,load_label_vectors_simple,load_concept2vec_vectors_simple],['cem','tcav','label','concept2vec']):
    for dataset in [CUB_Dataset(),MNIST_Dataset(),Chexpert_Dataset(),DSprites_Dataset()]:
        a = dataset.get_attributes() 

        similarities = []
        avg_same_similarity = []
        stds = []

        for seed in [43,44,45]:
            vectors = [np.mean(method(attribute,dataset,"",seed),axis=0) for attribute in a]
            for i in vectors:
                stds.append(np.std(i))
            cosine_similarities_max = []
            for i in range(len(vectors)):
                cosine_similarities = max([1-cosine(vectors[i],vectors[j]) for j in range(len(vectors)) if i!=j])
                cosine_similarities_max.append(cosine_similarities)
            similarities.append(np.mean(cosine_similarities_max))

        d = len(vectors[0])
        std = np.mean(stds)
        mean = np.mean(similarities)
        z_score = (mean-0)/(d*std**4/(3**.5))
        z_score *= len(attribute)**.5

        results_by_method[name][dataset.experiment_name] = {
            'dimension': d,
            'std': float(std), 
            'mean_similarity': float(mean), 
            'std_similarity': float(np.std(similarities))
        }
json.dump(results_by_method,open('results/evaluation/ablation/randomness_cem_tcav.json','w'))

In [ ]:
results_by_method['tcav']['cub']

In [ ]:
CUB_Dataset().get_attributes()

In [ ]:
load_concept2vec_vectors_simple("5_color",MNIST_Dataset(),"",43).shape

In [ ]:
create_concept2vec(MNIST_Dataset(),"",43)

In [ ]:
load_concept2vec_vectors_simple

In [ ]:
results_by_method['label']['mnist']['z_score']

## Analyze impact of vector metric

In [ ]:
hierarchy_by_metric = {}
dataset = CUB_Dataset()
attributes = dataset.get_attributes()
for metric in ['euclidean','cosine','manhattan']:
    hierarchy_by_metric[metric] = {}

    for function,name in zip([load_label_vectors_simple,
    load_shapley_vectors_simple, 
    load_cem_vectors_simple,
    load_concept2vec_vectors_simple
    ],['label','shapley','cem','concept2vec']):
        hierarchy_by_metric[metric][name] = {}
        for seed in [43,44,45]:
            hierarchy_by_metric[metric][name][seed] = flat_distance_to_square(get_concept_distances(function,dataset,'',attributes,seed,metric=metric))

In [ ]:
avg_pairwise_distance = {}
for name in ['label','shapley','cem','concept2vec']:
    avg_pairwise_distance[name] = {}
    for metric_1 in ['euclidean','cosine','manhattan']:
        avg_pairwise_distance[name][metric_1] = {}
        for metric_2 in ['euclidean','cosine','manhattan']:
            h1 = hierarchy_by_metric[metric_1][name]
            h2 = hierarchy_by_metric[metric_2][name]
            results = [embedding_distance(h1[seed],h2[seed],k=3) for seed in [43,44,45]]
            avg_pairwise_distance[name][metric_1][metric_2] = (np.mean(results),np.std(results))

In [ ]:
json.dump(avg_pairwise_distance,open('results/evaluation/ablation/metric_distances.json','w'))

In [ ]:
hierarchy_object_by_metric = {}
dataset = CUB_Dataset()
attributes = dataset.get_attributes()
for metric in ['euclidean','cosine','manhattan']:
    hierarchy_object_by_metric[metric] = {}

    for function,name in zip([load_label_vectors_simple,
    load_shapley_vectors_simple, 
    load_cem_vectors_simple,
    load_concept2vec_vectors_simple
    ],['label','shapley','cem','concept2vec']):
        hierarchy_object_by_metric[metric][name] = {}
        for seed in [43,44,45]:
            hierarchy_object_by_metric[metric][name][seed] = create_hierarchy(create_ward_hierarchy,function,dataset,'',attributes,seed,metric=metric) 

### Investigate why this occurs

In [ ]:
cosine_hierarchy = hierarchy_object_by_metric['cosine']['label'][43]
euclidean_hierarchy = hierarchy_object_by_metric['euclidean']['label'][43]

In [ ]:
top_k_euclidean = get_top_k_pairs(hierarchy_by_metric['euclidean']['label'][43],k=3)
top_k_cosine = get_top_k_pairs(hierarchy_by_metric['cosine']['label'][43],k=3)

top_k_euclidean = [(attributes[i[0]],attributes[i[1]]) for i in top_k_euclidean]
top_k_cosine = [(attributes[i[0]],attributes[i[1]]) for i in top_k_cosine]

In [ ]:
all_distances = [np.mean([embedding_distance(hierarchy_by_metric['cosine']['label'][seed],hierarchy_by_metric['euclidean']['label'][seed],k=k) for seed in [43,44,45]]) for k in range(1,110)]

In [ ]:
percent_euclidean_agree = len([i for i in top_k_euclidean if i[0].split("::")[1] == i[1].split("::")[1]])/len(top_k_euclidean)
percent_cosine_agree = len([i for i in top_k_cosine if i[0].split("::")[1] == i[1].split("::")[1]])/len(top_k_euclidean)
percent_euclidean_agree,percent_cosine_agree

In [ ]:
all_top_k_euclidean = []
all_top_k_cosine = []

for seed in [43,44,45]:
    top_k_euclidean = get_top_k_pairs(hierarchy_by_metric['euclidean']['label'][seed],k=3)
    top_k_cosine = get_top_k_pairs(hierarchy_by_metric['cosine']['label'][seed],k=3)

    top_k_euclidean = [(attributes[i[0]],attributes[i[1]]) for i in top_k_euclidean]
    top_k_cosine = [(attributes[i[0]],attributes[i[1]]) for i in top_k_cosine]

    all_top_k_euclidean += top_k_euclidean
    all_top_k_cosine += top_k_cosine


percent_euclidean_agree = len([i for i in all_top_k_euclidean if i[0].split("::")[1] == i[1].split("::")[1]])/len(top_k_euclidean)
percent_cosine_agree = len([i for i in all_top_k_cosine if i[0].split("::")[1] == i[1].split("::")[1]])/len(top_k_euclidean)
percent_euclidean_agree/3, percent_cosine_agree/3

In [ ]:
json.dump({
    'all_distances': all_distances, 
    'top_k_euclidean': top_k_euclidean, 
    'top_k_cosine': top_k_cosine, 
    'percent_euclidean_agree': percent_euclidean_agree/3, 
    'percent_cosine_agree': percent_cosine_agree/3, 
},open('results/evaluation/ablation/distance_cosine_euclidean_top_k.json','w'))

## Analyze Hierarchy Similairty

In [ ]:
hierarchy_by_dataset = defaultdict(lambda: defaultdict(dict))
for dataset_function,dataset_name in zip([CUB_Dataset, MNIST_Dataset, DSprites_Dataset, Chexpert_Dataset],['cub','mnist','dsprites','chexpert']):
    dataset = dataset_function()
    attributes = dataset.get_attributes()

    for function,name in zip([load_label_vectors_simple,
    load_shapley_vectors_simple, 
    load_cem_vectors_simple,
    load_concept2vec_vectors_simple
    ],['label','shapley','cem','concept2vec']):
        hierarchy_by_metric[metric][name] = {}
        for seed in [43,44,45]:
            hierarchy_by_dataset[dataset_name][name][seed] = flat_distance_to_square(get_concept_distances(function,dataset,'',attributes,seed))


In [ ]:
distance_by_dataset = defaultdict(lambda: defaultdict(dict))

for dataset in ['cub','mnist','dsprites','chexpert']:
    for name in ['cem','shapley','label','concept2vec']:
        for name_2 in ['cem','shapley','label','concept2vec']:
            h1 = hierarchy_by_dataset[dataset][name]
            h2 = hierarchy_by_dataset[dataset][name_2]
            distance_by_dataset[dataset][name][name_2] = [embedding_distance(h1[seed],h2[seed],k=3) for seed in [43,44,45]]


In [ ]:
distances_cub = np.array([[distance_by_dataset['cub'][i][j] for j in distance_by_dataset['cub'][i]] for i in distance_by_dataset['cub']])
distances_cub = np.mean(distances_cub,axis=2)

In [ ]:
labels = ['cem','shapley','label','concept2vec']

In [ ]:
json.dump(distances_cub.tolist(), open('results/evaluation/ablation/distance_between_hierarchies.json','w'))

## CUB Ablation Studies

### Evaluation at different noise levels

In [ ]:
dataset = CUB_Dataset()
attributes = dataset.get_attributes()

In [ ]:
embedding_method = load_shapley_vectors_simple
name = "Shapley"
random_seeds = [43]

In [ ]:
start = time.time()

results = {}

for flip_prob in ['0.01','0.05','0.1']:
    results['flip_prob_{}'.format(flip_prob)] = compare_same_images_by_suffix(embedding_method,
                                         dataset,attributes,random_seeds,"_flip_{}".format(flip_prob),
                                        baseline_hierarchies=None)

for noise in [25,50,100]:
    results['noise_{}'.format(flip_prob)] = compare_same_images_by_suffix(embedding_method,
                                         dataset,attributes,random_seeds,"_noise_{}".format(noise),
                                        baseline_hierarchies=None)

w = open("results/evaluation/cub_noise_ablation.txt","w")
for key in results:
    w.write("{}: {}\n".format(key,results[key]))
w.close()

### See how truthfulness hyperparameters impact things

In [ ]:
dataset = CUB_Dataset()
attributes = dataset.get_attributes()

results = {}

for compare_concept in [1,3,5,7]:
    results[compare_concept] = truthfulness_metric_shapley(load_shapley_vectors_simple,dataset,attributes,
                                                           [43,44,45],model_name="VGG16",compare_concepts=compare_concept)

In [ ]:
w = open("results/evaluation/cub_truthfulness_ablation.txt","w")
for key in results:
    w.write("{}: {}\n".format(key,results[key]))
w.close()